# Installation

In [1]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_WRITE_TOKEN')
!huggingface-cli login --add-to-git-credential --token $HF_TOKEN
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

Token is valid (permission: write).
The token `WriteToken` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
%%bash
pip install  --upgrade\
    'vllm>=0.8.2' \
    'transformers>=4.50.3' \
    pyzmq \
    unsloth \
    accelerate \
    bitsandbytes \
    openai \
    langchain-text-splitters \
    peft \
    FlagEmbedding \
    datasets \
    faiss-cpu \
    langchain-text-splitters \
    tavily-python \
    "flashinfer-python>=0.2.4"  --extra-index-url https://flashinfer.ai/whl/cu124/torch2.6/
git clone https://github.com/ggml-org/llama.cpp.git
cd llama.cpp/gguf-py/ && pip install --editable .
pip install jupyter-kernel-gateway ipykernel
# pip install --upgrade --no-deps numpy==1.26.4 pandas==2.2.2

Looking in indexes: https://pypi.org/simple, https://flashinfer.ai/whl/cu124/torch2.6/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.2/544.2 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.1/294.1 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
cudf-cu12 25.2.1 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
distributed-ucxx-cu12 0.42.0 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
dask-cuda 25.2.0 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
cuml-cu12 25.2.1 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61

```bash
VLLM_FLASH_ATTN_VERSION=3 \
VLLM_ATTENTION_BACKEND=FLASH_ATTN VLLM_USE_V1=1 \
VLLM_ALLOW_LONG_MAX_MODEL_LEN=1 \
TOKENIZERS_PARALLELISM=true MAX_JOBS=2 \
    vllm serve \
        ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g \
        --port 8877 \
        --max-model-len 2048 \
        --api-key token-abc123 \
        --quantization compressed-tensors \
        --max-num-seqs 1
```

```bash
mkdir /content/gguf/openfree
huggingface-cli download openfree/Mistral-Small-3.1-24B-Instruct-2503-Q8_0-GGUF --local-dir /content/gguf/openfree/Mistral-Small-3.1-24B-Instruct-2503-Q8_0-GGUF
VLLM_FLASH_ATTN_VERSION=2 \
VLLM_ATTENTION_BACKEND=FLASH_ATTN \
VLLM_USE_V1=0 \
VLLM_ALLOW_LONG_MAX_MODEL_LEN=1 \
TOKENIZERS_PARALLELISM=true \
MAX_JOBS=2 \
vllm serve \
    /content/gguf/openfree/Mistral-Small-3.1-24B-Instruct-2503-Q8_0-GGUF/mistral-small-3.1-24b-instruct-2503-q8_0.gguf \
        --port 8877 \
        --max-model-len 4096 \
        --api-key token-abc123 \
        --quantization gguf \
        --max-num-seqs 1
```

```bash
mkdir /content/gguf/google
huggingface-cli download google/gemma-3-4b-it-qat-q4_0-gguf --local-dir /content/gguf/google/gemma-3-4b-it-qat-q4_0-gguf
VLLM_FLASH_ATTN_VERSION=2 \
VLLM_ATTENTION_BACKEND=FLASH_ATTN \
VLLM_USE_V1=0 \
VLLM_ALLOW_LONG_MAX_MODEL_LEN=1 \
TOKENIZERS_PARALLELISM=true \
MAX_JOBS=2 \
vllm serve \
    /content/gguf/google/gemma-3-4b-it-qat-q4_0-gguf/gemma-3-4b-it-q4_0.gguf \
        --port 8877 \
        --max-model-len 4096 \
        --api-key token-abc123 \
        --quantization gguf \
        --max-num-seqs 1
```

# Web Search Agent

In [5]:
from tavily import TavilyClient
import asyncio, os, requests, time, json
from IPython.display import display, Markdown, Latex

tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [6]:
from openai import OpenAI
import math
import time
import json

client = OpenAI(
    base_url="http://localhost:8877/v1",
    api_key="token-abc123",
)

In [204]:
# prompt: localhost:8877/v1/models 에 Get 요청 보내서 결과 보는 코드 api 키는 token-abc123

import requests
import json

def get_model_info(url, api_key):
    headers = {
        "Authorization": f"Bearer {api_key}"
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching model information: {e}")
        return None

# Example usage
url = "http://localhost:8877/v1/models"
api_key = "token-abc123"

model_info = get_model_info(url, api_key)
model_name = ""
if model_info:
    model_name = model_info.get('data')[0].get('id')
else:
    raise Exception("vLLM Server is not running...")
print(json.dumps(model_info, indent=2))
print(model_name)

{
  "object": "list",
  "data": [
    {
      "id": "google/gemma-3-1b-it",
      "object": "model",
      "created": 1745802940,
      "owned_by": "vllm",
      "root": "google/gemma-3-1b-it",
      "parent": null,
      "max_model_len": 8192,
      "permission": [
        {
          "id": "modelperm-277ea8a55310445b88661c8f1c88a8d8",
          "object": "model_permission",
          "created": 1745802940,
          "allow_create_engine": false,
          "allow_sampling": true,
          "allow_logprobs": true,
          "allow_search_indices": false,
          "allow_view": true,
          "allow_fine_tuning": false,
          "organization": "*",
          "group": null,
          "is_blocking": false
        }
      ]
    }
  ]
}
google/gemma-3-1b-it


In [8]:
def deduplicate_and_format_sources(search_response, max_tokens_per_source, include_raw_content=True):
     # Collect all results
    sources_list = []
    for response in search_response:
        sources_list.extend(response['results'])

    # Deduplicate by URL
    unique_sources = {source['url']: source for source in sources_list}

    # Format output
    formatted_text = "Content from sources:\n"
    for i, source in enumerate(unique_sources.values(), 1):
        formatted_text += f"{'='*80}\n"  # Clear section separator
        formatted_text += f"Source: {source['title']}\n"
        formatted_text += f"{'-'*80}\n"  # Subsection separator
        formatted_text += f"URL: {source['url']}\n===\n"
        formatted_text += f"Most relevant content from source: {source['content']}\n===\n"
        if include_raw_content:
            # Using rough estimate of 4 characters per token
            char_limit = max_tokens_per_source * 4
            # Handle None raw_content
            raw_content = source.get('raw_content', '')
            if raw_content is None:
                raw_content = ''
                print(f"Warning: No raw_content found for source {source['url']}")
            if len(raw_content) > char_limit:
                raw_content = raw_content[:char_limit] + "... [truncated]"
            formatted_text += f"Full source content limited to {max_tokens_per_source} tokens: {raw_content}\n\n"
        formatted_text += f"{'='*80}\n\n" # End section separator

    return formatted_text.strip()

In [9]:
def generate_response(message_list):
    completion = client.chat.completions.create(
        model = model_name,
        messages = message_list,
        max_tokens=2048,
        frequency_penalty=0.3,
        temperature=0.6,
        stream=True,
    )

    final_answer = []
    assistant_response = ""

    start = time.time()

    # 스트림 모드에서는 completion.choices 를 반복문으로 순회
    for chunk in completion:
        chunk_content = chunk.choices[0].delta.content

        if isinstance(chunk_content, str):
            final_answer.append(chunk_content)
            # 토큰 단위로 실시간 답변 출력
            print(chunk_content, end="")
            assistant_response += chunk_content

    end = time.time()
    print(f"\n\ninference time: {end - start:.5f} sec \n\n")
    return assistant_response

In [10]:
import threading

def worker(query, search_result, req_num_result, include_raw, req_topic):
    print(f"Thread: {query}")
    search_result.append(
        tavily_client.search(
            query,
            max_results= req_num_result,
            include_raw_content= include_raw,
            topic= req_topic
        )
    )

In [11]:
def ask_tavily(search_queries, search_tasks, req_num_result, include_raw, req_topic):
    start_time = time.time()
    threads = []

    for query in search_queries:
        t = threading.Thread(target=worker, args=(query, search_tasks, req_num_result, include_raw, req_topic))
        threads.append(t)
        t.start()

    for thread in threads:
        thread.join()

    end_time = time.time()
    execution_time = end_time - start_time

    print(f"\nask_tavily task running time: {execution_time:.2f}초 \n")

In [12]:
def ask_plan_query_writer(topic, content):
    llm_prompt = """You are an expert technical writer crafting a section that synthesizes information
<section topic>
""" + topic + """
</section topic>

<section organization>
""" + content + """
</section organization>

<Task>
Your goal is to generate 3 web search queries that will help gather information for planning the sections.

The queries should:

1. Be related to the section topic
2. Help satisfy the requirements specified in the section organization

Make the queries specific enough to find high-quality, relevant sources while covering the breadth needed for the section structure.

Note1. that today's date is """+time.strftime("%Y-%m-%d")+""".
Note2. Output your response in JSON format, with the following structure: { "queries": [ "query1", "query2", "query3" ] }
Only output in JSON format when generating responses. Never include additional phrases such as "here is content in JSON format".
</Task>"""

    return llm_prompt

In [13]:
def ask_final_writer_instructions(topic, content, search_tasks):
    final_section_writer="""You are an expert technical writer.

<Section name>
""" + content + """
</Section name>

<Section topic>
""" + topic + """
</Section topic>

<Available Website Search Content>
""" + deduplicate_and_format_sources(search_tasks, max_tokens_per_source=4000, include_raw_content=True) + """
</Available Website Search Content>

<Task>
1. Section-Specific Approach:

For Introduction:
- Use # for Website Search title (Markdown format)
- Write in simple and clear language
- Focus on the core motivation for the Section in 1-2 paragraphs
- Use a clear narrative arc to introduce the Section
- Include NO structural elements (no lists or tables)
- No sources section needed

For Conclusion/Summary:
- Use ## for Conclusion/Summary title (Markdown format)
- For comparative Conclusion/Summary:
    * Must include a focused comparison table using Markdown table syntax
    * Table should distill insights from the Section
    * Keep table entries clear and concise
- For non-comparative Conclusion/Summary:
    * Only use ONE structural element IF it helps distill the points made in the Section:
    * Either a focused table comparing items present in the Section (using Markdown table syntax)
    * Or a short list using proper Markdown list syntax:
      - Use `*` or `-` for unordered lists
      - Use `1.` for ordered lists
      - Ensure proper indentation and spacing
- Sources and url section needed. (especially when expressing a URL, please provide the entire URL exactly as given in the content without abbreviating it.)
- End with specific next steps or implications

2. Writing Approach:
- Use concrete details over general statements
- Make every word count
- Focus on your single most important point
</Task>

<Quality Checks>
- Verify that EVERY claim is grounded in the provided Source material
- Confirm each URL appears ONLY ONCE in the Source list
- For introduction: # for Website Search title, no structural elements, no sources section
- For conclusion: ## for Conclusion/Summary title, only ONE structural element at most, add sources and url section
- Markdown format
- Do not include word count or any preamble in your response
</Quality Checks>

Please note that respond in Korean always."""

    return final_section_writer

In [14]:
system_prompt = "You are a helpful assistant. "

topic = "판례확인"
content = "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?"

messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": ask_plan_query_writer(topic, content)},
    ]

response_query = generate_response(messages)

```json
{
  "queries": [
    "local government official misconduct supervisor liability 지방공무원 비위 감독 책임",
    "지방공무원법 부하직원 비위 감독 의무 및 책임 지방공무원법 부하직원 비위 감독 의무 및 책임",
    "지방공무원법 부하직원 비위 감독자 책임 범위 지방공무원법 부하직원 비위 감독자 책임 범위"
  ]
}
```

inference time: 4.30529 sec 




In [15]:
if "json" in response_query:
    response_query = response_query.split("```json")[1].split("```")[0]
json_data = json.loads(response_query)
queries = json_data['queries']

print("사용자 발화 기반으로 추출한 web query 문장 3건:")
print(queries)

search_tasks = []
req_topic = 'general' # news   gerneral 과 news 중 선택
req_num_result = 3    # 각 web query 에 대해 리턴할 site 개수
include_raw = False    # site 의 원본 컨텐츠 리턴 유무

ask_tavily(queries, search_tasks, req_num_result, include_raw, req_topic)
print(search_tasks)

사용자 발화 기반으로 추출한 web query 문장 3건:
['local government official misconduct supervisor liability 지방공무원 비위 감독 책임', '지방공무원법 부하직원 비위 감독 의무 및 책임 지방공무원법 부하직원 비위 감독 의무 및 책임', '지방공무원법 부하직원 비위 감독자 책임 범위 지방공무원법 부하직원 비위 감독자 책임 범위']
Thread: local government official misconduct supervisor liability 지방공무원 비위 감독 책임
Thread: 지방공무원법 부하직원 비위 감독 의무 및 책임 지방공무원법 부하직원 비위 감독 의무 및 책임
Thread: 지방공무원법 부하직원 비위 감독자 책임 범위 지방공무원법 부하직원 비위 감독자 책임 범위

ask_tavily task running time: 2.34초 

[{'query': '지방공무원법 부하직원 비위 감독 의무 및 책임 지방공무원법 부하직원 비위 감독 의무 및 책임', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': '상사는 부하직원 비위행위에 성실의무 책임 있는지? : 네이버 블로그', 'url': 'https://m.blog.naver.com/pdsph1004/222061319284', 'content': '8. 22., 선고, 78누164, 판결] 지방공무원의 징계에 관한 어느 법규에도 부하직원에게 파면에 해당하는 비위사실이 있는 경우에는 직접 감독 책임이 있는 공무원에게 감독 의무를 위반한 것으로 본다는 규정이 없으므로 아무리 부하직원에게 파면에 해당하는 비위사실이나 또는 그 이상의 위법된 행위가 있음이 인정된다 하더라도 이를 감독할 책임이 있는 공무원에게 구체적인 감독 의무를 위반한 사실이 증거에 의하여 인정되지 아니하는 이상 비위사실이나 위법행위를 한 공무원이 있는 경우에는 그 감독자를 당연히 감독 의무

In [16]:
search_tasks

[{'query': '지방공무원법 부하직원 비위 감독 의무 및 책임 지방공무원법 부하직원 비위 감독 의무 및 책임',
  'follow_up_questions': None,
  'answer': None,
  'images': [],
  'results': [{'title': '상사는 부하직원 비위행위에 성실의무 책임 있는지? : 네이버 블로그',
    'url': 'https://m.blog.naver.com/pdsph1004/222061319284',
    'content': '8. 22., 선고, 78누164, 판결] 지방공무원의 징계에 관한 어느 법규에도 부하직원에게 파면에 해당하는 비위사실이 있는 경우에는 직접 감독 책임이 있는 공무원에게 감독 의무를 위반한 것으로 본다는 규정이 없으므로 아무리 부하직원에게 파면에 해당하는 비위사실이나 또는 그 이상의 위법된 행위가 있음이 인정된다 하더라도 이를 감독할 책임이 있는 공무원에게 구체적인 감독 의무를 위반한 사실이 증거에 의하여 인정되지 아니하는 이상 비위사실이나 위법행위를 한 공무원이 있는 경우에는 그 감독자를 당연히 감독 의무를 위반한 자로 볼 수는 없다. 26., 선고, 89누589, 판결] : 부하직원이 구체적으로 어떠한 직무수행상 태만이나 고의가 있었는지 구체적인 감독 의무 위반 사실을 밝혀 증거에 의하여 이를 인정하여야 한다.',
    'score': 0.8106142,
    'raw_content': None},
   {'title': '공무원의 성실의무 의미 및 조문 해설 - 네이버 블로그',
    'url': 'https://m.blog.naver.com/pdsph1004/222057242597',
    'content': '공무원의 성실의무 의미 및 조문 해설 ... 「 지방공무원법」제48조 ... ④ 감독자는 부하직원의 비위행위를 사전에 방지하는 노력도 성실의무에 포함한다. 즉, 부하직원에 대한 상시 감독 의무를 게을리하지 않아야 한다.',
    'score': 0.7043

In [17]:
print("\n\n=================================================================\n")
system_prompt = """You are a helpful assistant. And Answers must be in Korean. \
그리고 답변할땐 꼭 다음의 지시 사항을 준수해줘. \
1) 질문에 대한 답변 후보 문장들을 자세히 읽고 유저가 물어본 질문에 제시된 정보만 활용해서 질문과 정확성, 관련성, 신뢰성을 종합적으로 고려하여 답변을 만들어 제공해주세요. \
2) 후보문장은 json 형태로 돼 있습니다.
3) 후보문장 중 답변에 활용하는데 가장 적합한 후보가 어떤것 이였는지를 알려 주세요.\
4) 후보 문장들 중 질문에 대한 답이 없을 경우 "답변을 찾을 수 없습니다. 좀 더 구체적으로 질문해 주세요." 라고 답하세요."""

user_prompt =  content + "\n후보문장: " + str(search_tasks)

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

response_query = generate_response(messages)

print("\n\n=========================  Search Report  ========================================\n")
display(Markdown(response_query))




지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는가에 대한 질문에 대해, 감독자가 위법한 행위를 한 것인지 여부는 복잡한 문제입니다. 

가장 적합한 후보 문장은 다음과 같습니다:

*   **title:** 상사는 부하직원 비위행위에 성실의무 책임 있는지? : 네이버 블로그
    *   **url:** https://m.blog.naver.com/pdsph1004/222061319284
    *   **score:** 0.7171114
    *   **raw_content:** 8. 22., 선고, 78누164, 판결] 지방공무원의 징계에 관한 어느 법규에도 부하직원에게 파면에 해당하는 비위사실이 있는 경우에는 직접 감독 책임이 있는 공무원에게 감독 의무를 위반한 것으로 본다는 규정이 없으므로 아무리 부하직원에게 파면에 해당하는 비위사실이나 또는 그 이상의 위법된 행위가 있음이 인정된다 하더라도 이를 감독할 책임이 있는 공무원에게 구체적인 감독 의무를 위반한 사실이 증거에 의하여 인정되지 아니하는 이상 비위사실이나 위법행위를 한 공무원이 있는 경우에는 그 감독자를 당연히 감독 의무를 위반한 자로 볼 수는 없다. 26., 선고, 89누589, 판결] : 부하직원이 구체적으로 어떠한 직무수행상 태만이나 고의가 있었는지 구체적인 감독 의무 위반 사실을 밝혀 증거에 의하여 이를 인정하여야 한다.

이 문장은 명확하게 감독자가 부하직원의 비위행위에 대한 감독 의무를 위반했는지 여부에 대한 법적 판단을 제시합니다.  판례를 바탕으로, 단순히 비위사실이 존재하는 경우에만 감독자가 책임이 있는 것은 아니며, 구체적으로 감독 의무를 위반했다는 것을 증거로 입증해야 한다고 설명하고 있습니다.

답변: 지방공무원법상 부하직원의 비위사실이 있는 경우, 감독자가 반드시 위법한 행위를 한 것은 아닙니다. 부하직원의 비위행위에 대한 구체적인 감독 의무 위반 사실을 증거로 입증해야 책임을 물을 수 있습니다.

inference time: 19.16082 se

지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는가에 대한 질문에 대해, 감독자가 위법한 행위를 한 것인지 여부는 복잡한 문제입니다. 

가장 적합한 후보 문장은 다음과 같습니다:

*   **title:** 상사는 부하직원 비위행위에 성실의무 책임 있는지? : 네이버 블로그
    *   **url:** https://m.blog.naver.com/pdsph1004/222061319284
    *   **score:** 0.7171114
    *   **raw_content:** 8. 22., 선고, 78누164, 판결] 지방공무원의 징계에 관한 어느 법규에도 부하직원에게 파면에 해당하는 비위사실이 있는 경우에는 직접 감독 책임이 있는 공무원에게 감독 의무를 위반한 것으로 본다는 규정이 없으므로 아무리 부하직원에게 파면에 해당하는 비위사실이나 또는 그 이상의 위법된 행위가 있음이 인정된다 하더라도 이를 감독할 책임이 있는 공무원에게 구체적인 감독 의무를 위반한 사실이 증거에 의하여 인정되지 아니하는 이상 비위사실이나 위법행위를 한 공무원이 있는 경우에는 그 감독자를 당연히 감독 의무를 위반한 자로 볼 수는 없다. 26., 선고, 89누589, 판결] : 부하직원이 구체적으로 어떠한 직무수행상 태만이나 고의가 있었는지 구체적인 감독 의무 위반 사실을 밝혀 증거에 의하여 이를 인정하여야 한다.

이 문장은 명확하게 감독자가 부하직원의 비위행위에 대한 감독 의무를 위반했는지 여부에 대한 법적 판단을 제시합니다.  판례를 바탕으로, 단순히 비위사실이 존재하는 경우에만 감독자가 책임이 있는 것은 아니며, 구체적으로 감독 의무를 위반했다는 것을 증거로 입증해야 한다고 설명하고 있습니다.

답변: 지방공무원법상 부하직원의 비위사실이 있는 경우, 감독자가 반드시 위법한 행위를 한 것은 아닙니다. 부하직원의 비위행위에 대한 구체적인 감독 의무 위반 사실을 증거로 입증해야 책임을 물을 수 있습니다.

# Project

# [Mini Project]

## ㅇㅇㅇ 지능을 갖고있는 Agent 만들기

  - 오늘은 그간 실습했던 prompt engineering, Tool 활용, RAG, PEFT 등을 활용하여 각자 관심있는 영역에대해 질의응답할 수 있는 Agent를 만들어보겠습니다.
  - llama3.1 에게 LLM 기반 agent 개발할때 어떤 step 이 필요한지 확인해봐주세요.

---

In [18]:
from openai import OpenAI
import math
import time
import json

client = OpenAI(
    base_url="http://localhost:8877/v1",
    api_key="token-abc123",
)

def vllm_generate(system_prompt, user_prompt):
    completion = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "user", "content": system_prompt + '\n' + user_prompt}
        ] if 'gemma' in model_name.lower() else [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        frequency_penalty=0.09,
        temperature=0.3,
        top_p=0.95,
        max_tokens=1024,
        stream=True,
    )

    final_answer = []
    assistant_response = ""

    start = time.time()

    # 스트림 모드에서는 completion.choices 를 반복문으로 순회
    for chunk in completion:
        chunk_content = chunk.choices[0].delta.content

        if isinstance(chunk_content, str):
            final_answer.append(chunk_content)
            # 토큰 단위로 실시간 답변 출력
            print(chunk_content, end="")
            assistant_response += chunk_content

    end = time.time()
    print(f"\n\ninference time: {end - start:.5f} sec \n\n")

In [20]:
system_prompt = "You are a helpful assistant. And Answers must be in Korean."
user_prompt = "LLM을 활용한 Agent 를 개발할때 어떤 step 이 필요한지 개조식으로 설명해줘."

vllm_generate(system_prompt, user_prompt)

## LLM 기반 에이전트 개발 단계 (개조식)

LLM(Large Language Model)을 활용한 에이전트 개발은 복잡하지만, 체계적인 단계를 거치면 성공적인 결과물을 얻을 수 있습니다. 다음은 주요 단계들을 개조식으로 정리한 것입니다.

**1. 기획 및 정의 단계 (Planning & Definition)**

*   **1.1. 에이전트 목표 정의:**
    *   에이전트가 무엇을 해야 하는지 명확하게 정의합니다. (예: 고객 문의 응대, 상품 추천, 데이터 분석 보고서 작성 등)
    *   구체적인 목표를 설정해야 LLM의 성능을 평가하고 개선하기 용이합니다.
*   **1.2. 에이전트 역할 및 책임 설정:**
    *   에이전트가 어떤 역할을 수행할지, 어떤 책임을 지게 될지 정의합니다. (예: 상담원, 전문가, 조력자 등)
*   **1.3. 사용 시나리오 및 워크플로우 설계:**
    *   에이전트가 실제로 사용될 상황을 가정하고, 사용자의 요청에 따라 어떤 과정을 거쳐야 하는지 상세하게 설계합니다. (시나리오 작성)
*   **1.4. 필요한 데이터 파악:**
    *   에이전트가 작동하는 데 필요한 데이터 (데이터 종류, 양, 품질 등)를 파악합니다.

**2. LLM 선택 및 설정 단계 (LLM Selection & Setup)**

*   **2.1. LLM 선택:**
    *   목표와 예산에 맞는 LLM을 선택합니다. (GPT-3.5, GPT-4, Claude, PaLM 등)
    *   각 LLM의 장단점, 비용, API 사용 가능 여부 등을 고려합니다.
*   **2.2. LLM API 연동:**
    *   선택한 LLM의 API를 개발 환경에 연동합니다. (OpenAI API, Google Cloud AI Platform 등)
*   **2.3. 프롬프트 엔지니어링:**
    *   LLM에게 효과적인 지시를 내리기 위한 프롬프트(Prompt)를 설계합니다.
    *   명확하고 구체적인 프롬프

---
## 문제 정의 & 데이터 수집 및 전처리

  - Agent 개발 첫 step : 뭐니뭐니 해도 Data  
    [AI Hub] https://aihub.or.kr/
    huggingface 의 dataset

    국내데이터는 AI Hub 에도 전처리 잘돼 있는 데이터셋 들이 많이 있습니다.  
    ex) 법률 - 상황에 따른 판례 데이터
    https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=71723  

    각 판례가 json 형태로 정리돼 있는 데이터 입니다.

---

In [21]:
user_prompt = """교통계 순경의 파면사례에 대해 알려줘. 답변할땐 참고문장을 반드시 참고해서 답변해줘. \n
참고문장
{
  "판례상세링크": "/DRF/lawService.do?OC=pwshoot&target=prec&ID=93872&type=HTML&mobileYn=",
  "판시사항": "【판시사항】\n\n교통계순경이 주차금지 구역의 주차를 묵인하고 그 위반자로부터 이조회화액자 일점을 교부받은 비위사실을 징계사유로 한 파면처분이 징계재량권의 범위를 넘어선 것인지 여부",
  "판결요지": "【판결요지】\n\n교통계 순경 (갑)이 주차금지구역의 주차를 단속하지 않은 장소가 교통에 크게 지장을 주는 장소도 아니고 주차위반자인 (을)로부터 교부받았다는 이조회화액자 1점은 (을)의 소속회사에서 고객을 위하여 만든 특별 증정품에 불과하며 그리고 (갑)은 여러차례 표창을 받은 사실이 있는 점 등을 종합하면 위 비위사실만 가지고 (갑)에 대하여 징계처분 중 가장 무거운 파면처분을 택한 것은 징계재량권의 범위를 일탈하였다고 할 것이다.",
  "참조조문": "【참조조문】\n\n경찰공무원법 제53조 제1항 제1호\n\n제53조 제1항 제3호",
  "참조판례": null,
  "판례내용": "【전문】\n\n【원고, 상고인】 원고 소송대리인 변호사 이해우\n\n【피고, 피상고인】 내무부장관\n\n【원 판 결】 서울고등법원 1976.9.21. 선고 76구181 판결\n\n【주문】\n\n원판결을 파기하고 사건을 서울고등법원에 환송한다.\n\n【이유】\n\n원고 소송대리인의 상고이유(상고이유보충서 기재 사실은 법정기간내에 제출된 상고이유를 부언하는 한도내에서 참작한다)를 판단한다.\n\n원판결의 설시이유에 의하면, 원심은 원고가 ○○경찰서 경비과 교통계 순경으로 ○○△가 □□□ 앞 로타리에서 교통정리 및 위법차량 등의 단속업무에 종사하고 있었던 바, 전부터 안면이 있는 소외인이 원고의 업무구역내인 □□□ 옆 주차금지구역에 자주 주차하여도 원고는 위 소외인과의 안면을 생각하여 교통법규위반자로 적발하지 않았고 이에 위 소외인은 평소 원고에 대하여 미안한 생각이 있어 1975.10.14 15:55경 역시 위 □□□ 옆 골목에 위법주차 중 마침 그 곳을 지나가던 원고에게 동 소외인 근무회사가 고객을 위하여 특별증정품으로 만들어 둔 이조회화 액자 1점을 교부한 사실을 인정한 후, 원고의 위와 같은 비위는 경찰공무원법 제53조 제1항 제1호및 제3호소정의 징계사유에 해당하는 것이며 이건 파면처분이 징계재량권의 범위를 넘어선 것이라고는 볼 수 없다고 판시하고 있다.\n\n살피건대 원판결이 들고 있는 증거들을 기록에 비추어 검토하여 보면 원판결이 그 판시와 같이 징계사유에 해당하는 비위사실이 있음을 인정한 과정에는 아무런 위법사유가 있다고 할 수 없다. 그러니 원판결에 심리미진으로 인한 사실오인이 있다는 상고논지 제1점은 이유없다.\n\n그러나원고가 여러차례 표창을 받은 사실과 원고가 받았다는 위 액자가 위 소외인 소속회사에서 고객을 위하여 만든 특별 증정품에 불과한 것임은 원심도 인정하고 있는 바이고 또 본건 단속대상 차량이 위법 주차한 장소가 교통에 크게 지장을 주는 장소임을 긍인하게 할 자료를 기록상 찾아 볼 수 없는 본건에 있어서, 원고에 대하여 징계처분 중 가장 무거운 파면처분을 택한 것은 징계재량권의 범위를 일탈한 것이라 할 것이다.그렇다면 원판결은 심리미진으로 인한 이유불비 내지는 징계재량권의 범위에 관한 법리를 오해한 위법을 범한 것이라고 할 것이니 이 점에서 원판결은 파기를 면하지 못할 것이고 이점을 지적하는 상고 논지 제2점은 이유있다.\n\n그러므로 원판결을 파기하고 사건을 원심법원에 환송하기로 하여 관여법관의 일치된 의견으로 주문과 같이 판결한다.\n\n대법관   한환진(재판장) 민문기 안병수 강안희"
}"""

vllm_generate(system_prompt, user_prompt)

네, 교통계 순경의 파면 사례에 대해 자세히 알려드리겠습니다.

**교통계 순경 파면 사례: 주차금지 구역 묵인 및 이조회화 액자 수령**

위 판례에서 언급된 사례는 교통계 순경(갑)이 주차금지 구역의 주차를 묵인하고, 위반자로부터 이조회화 액자 1점을 교부받은 비위로 파면 처분을 받은 사건입니다.

**사건의 핵심 내용:**

*   **비위 사실:** 갑 순경은 주차금지 구역의 주차를 단속하지 않았고, 위반자인 을(주차 위반자)로부터 이조회화 액자 1점을 교부받았습니다. 이 액자는 을의 회사에서 고객을 위해 특별히 제작된 증정품이었습니다.
*   **법적 근거:** 경찰공무원법 제53조 제1항 제1호 및 제3호에 따라 징계 사유에 해당한다고 판단되었습니다.
*   **파면 처분의 문제점:** 법원은 갑 순경이 여러 차례 표창을 받은 점, 이조회화 액자가 특별 증정품이라는 점, 해당 주차 구역이 교통에 큰 지장을 주지 않는다는 점 등을 종합적으로 고려하여 파면 처분이 징계 재량권을 일탈한 것으로 판단했습니다.

**판결의 주요 내용:**

*   **징계 재량권의 범위:** 법원은 갑 순경의 비위 사실만으로는 파면 처분이 징계 재량권의 범위를 넘어서는 것이 아니라고 보았습니다. 하지만, 위와 같은 여러 요소를 고려할 때, 파면 처분은 징계 재량권을 일탈한 처분이라고 판단했습니다.
*   **환송:** 서울고등법원은 원판결을 파기하고 사건을 다시 서울고등법원에 환송하기로 결정했습니다.

**참고 사항:**

*   이 사례는 징계 재량권의 범위와 징계 수위 결정 시 고려해야 할 다양한 요소를 보여주는 중요한 판례입니다.
*   교통계 순경은 교통 법규 위반 차량 단속을 통해 교통 안전을 확보하는 중요한 역할을 수행해야 합니다. 따라서 주차금지 구역 묵인 등 비위 행위는 엄중하게 다루어져야 합니다.

**추가적으로 알아두면 좋은 점:**

*   경찰공무원법은 공무원이 직무를 수행하면서 저지른 비위 행위에 대한 징계를 규정하고 있습니다. 징계 수위는 위반 행위

In [22]:
user_prompt="linux 디렉토리에 있는 여러개의 json 파일을 하나씩 pyhon 으로 읽어들여 text list 형태로 저장하는 코드 알려줘."
vllm_generate(system_prompt, user_prompt)

네, Linux 디렉토리에 있는 여러 개의 JSON 파일을 하나씩 Python으로 읽어들여 텍스트 리스트 형태로 저장하는 코드입니다.

```python
import os
import json

def json_파일_읽어_텍스트_리스트_생성(디렉토리_경로):
  """
  지정된 디렉토리의 모든 JSON 파일을 읽어들여 텍스트 리스트 형태로 반환합니다.

  Args:
    디렉토리_경로: JSON 파일이 있는 디렉토리의 경로입니다.

  Returns:
    JSON 파일 내용을 담은 텍스트 리스트입니다.
  """

  텍스트_리스트 = []
  for 파일_이름 in os.listdir(디렉토리_경로):
    if 파일_이름.endswith(".json"):
      파일_경로 = os.path.join(디렉토리_경로, 파일_이름)
      try:
        with open(파일_경로, 'r', encoding='utf-8') as f:  # encoding 지정 중요!
          데이터 = json.load(f)
          텍스트_리스트.append(json.dumps(데이터, ensure_ascii=False, indent=4)) # JSON 문자열로 변환 및 들여쓰기
      except Exception as e:
        print(f"파일 읽기 오류: {파일_경로} - {e}")
  return 텍스트_리스트

# 사용 예시
디렉토리_경로 = "/path/to/your/directory"  # 실제 디렉토리 경로로 변경하세요.
결과_리스트 = json_파일_읽어_텍스트_리스트_생성(디렉토리_경로)

for i, 내용 in enumerate(결과_리스트):
  print(f"파일 {i+1}:")
  print(내용)
  print("-" * 20)
```

**코드 설명:**

1.  **`import os`**: 파일 시스템과 상호 작용하기 위한 모듈을 가져옵니다.
2.  **`import json`**: 

In [23]:
import json
import os

# 디렉토리 경로를 입력하세요
directory_path = 'data_test/'
text_list = []

# 디렉토리 내 모든 JSON 파일을 읽어들입니다.
for filename in os.listdir(directory_path):
    if filename.endswith(".json"):  # JSON 파일만 읽어들입니다.
        file_path = os.path.join(directory_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            # JSON 데이터를 텍스트 리스트 형태로 변환합니다.
            text_json = json.dumps(data, indent=4, ensure_ascii=False)
            text_list.append(text_json)
    else:
        continue


In [24]:
text_list[0]

'{\n    "판례일련번호": 118463,\n    "사건명": "양수금",\n    "사건번호": "88가합21300",\n    "선고일자": "1988-11-15",\n    "법원명": "서울민사지방법원",\n    "법원종류코드": null,\n    "사건종류명": "민사",\n    "사건종류코드": 400101.0,\n    "판결유형": "제10부판결 : 항소",\n    "선고": "선고",\n    "판례상세링크": "/DRF/lawService.do?OC=pwshoot&target=prec&ID=118463&type=HTML&mobileYn=",\n    "판시사항": "【판시사항】\\n\\n도산한 회사의 채권을 근로자들이 임금채권을 확보하기 위하여 양수한 경우, 그 양수금채권이 체납국세보다 우선하는지 여부(적극)",\n    "판결요지": "【판결요지】\\n\\n도산한 갑회사의 근로자들이 임금채권을 변제받기 위한 방법으로 위 근로자들의 대표가 갑으로부터 갑의 을에 대한 중도금반환채권을 양수하여 을을 상대로 양수금청구를 하는 경우, 비록 을에게 위 채권양도의 통지가 도달하기 전에 국가가 갑에 대한 국세체납절차에서 갑의 을에 대한 위 중도금반환채권의 압류통지를 을에게 하였다 할지라도 위 근로자들은 그 압류통지에 불구하고 우선하여 위 양수금청구를 할 수 있다.",\n    "참조조문": "【참조조문】\\n\\n근로기준법 제30조의2",\n    "참조판례": null,\n    "판례내용": "【전문】\\n\\n【원고】 원고 1 외 1인\\n\\n【피고】 중소기업은행\\n\\n【주문】\\n\\n1. 피고는 원고들에게 금 21,680,911원 및 이에 대한 1988.6.8.부터 같은 해 11.15.까지는 연 5푼의, 그 다음날부터 다 갚는 날까지는 연 2할 5푼의 각 비율에 의한 금원을 지급하라.\\n\\n2. 원고들의 나머지 청구를 기각한다.\\n\\n3. 소송비용은 피고의 부담으로 한다.\\n\\n【청구취지】\\n\\n피고는 원고들에게 

In [25]:
directory_path = 'data_test/'
data_list = []
count = 0

# 디렉토리 내 모든 JSON 파일을 읽어들입니다.
for filename in os.listdir(directory_path):
    if filename.endswith(".json"):  # JSON 파일만 읽어들입니다.
        file_path = os.path.join(directory_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            # JSON 데이터를 텍스트 리스트 형태로 변환합니다.
            text_json = json.dumps(data, indent=4, ensure_ascii=False)
            data_list.append(text_json)
            if count % 10 ==0:
                print(". ", end="")
            count += 1
    else:
        continue

. . . . . . . . . . . . . . . 

In [26]:
len(data_list)

150

---
# RAG 준비  
## BGE-M3 모델로 준비된 데이터셋을 embedding 합니다.  
  - 현재 GPU instance 에서 nvidia-smi 를 수행해봐주세요 (gpu mem 확인)
  - vllm 으로 모델 서빙하고 있던것이 있다면 잠시 vllm 은 내려주세요.
      
mem 확보 후 BGE-Me 모델을 gpu 에 로드합니다.  

---

In [27]:
from transformers import AutoTokenizer, AutoModel
import torch, os, faiss
from FlagEmbedding import FlagModel
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")
raw_model = AutoModel.from_pretrained("BAAI/bge-m3")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

In [28]:
# get the BGE embedding model
model = FlagModel(
    'BAAI/bge-m3',
    query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
    use_fp16=True)

In [30]:
# get the embedding of the corpus
law_embeddings = model.encode(data_list)

print("shape of the corpus embeddings:", law_embeddings.shape)
print("data type of the embeddings: ", law_embeddings.dtype)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


shape of the corpus embeddings: (150, 1024)
data type of the embeddings:  float16


---
## embedding 완료 후 faiss index 로 embedding 등록  
  - law_embedding 을 바로 사용하면 안되는건가?

---

In [31]:
law_embeddings_np = law_embeddings.astype(np.float32)
dim = law_embeddings_np.shape[-1]

# create the faiss index and store the corpus embeddings into the vector space
index = faiss.index_factory(dim, 'Flat', faiss.METRIC_INNER_PRODUCT)

In [32]:
print(index.is_trained)

# add all the vectors to the index
index.add(law_embeddings_np)

print(f"total number of vectors: {index.ntotal}")

True
total number of vectors: 150


In [33]:
queries = [
    "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?",
]

query_embeddings = model.encode_queries(queries)
dists, ids = index.search(query_embeddings.astype(np.float32), k=3)

print(f"query:\t{queries[0]}\n\n")
count = 0
for i in range(3):
    print(f"RAG_{count}:\t{data_list[ids[0][i]]}\n")
    count = count+1

query:	지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?


RAG_0:	{
    "판례일련번호": 93952,
    "사건명": "감봉처분취소",
    "사건번호": "78누164",
    "선고일자": "1978-08-22",
    "법원명": "대법원",
    "법원종류코드": null,
    "사건종류명": "일반행정",
    "사건종류코드": 400107.0,
    "판결유형": "판결",
    "선고": "선고",
    "판례상세링크": "/DRF/lawService.do?OC=pwshoot&target=prec&ID=93952&type=HTML&mobileYn=",
    "판시사항": "【판시사항】\n\n지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 당연히 감독의무를 위반하였다고 볼 수 있는지 여부",
    "판결요지": "【판결요지】\n\n지방동무원의 징계에 관한 어느 법규에도 부하직원에게 징계사유가 있다 하여 감독자가 감독의무를 위반한 것으로 본다는 규정이 없고 또 서울시의 지방공무원 징계의 양정에 관한 규칙의 취지도 감독자에게 지방공무원법상의 징계사유가 있음을 전체로 규정된 것이라 할 것이다",
    "참조조문": "【참조조문】\n\n서울특별시지방공무원징계의양정에관한규칙 제2조 제1항, 제2조 제3항",
    "참조판례": null,
    "판례내용": "【전문】\n\n【원고, 상고인】 원고\n\n【피고, 피상고인】 서울특별시장 소송대리인 변호사 장영복\n\n【원 판 결】 서울고등법원 1978.3.28. 선고 77구225 판결\n\n【주문】\n\n원판결을 파기하고 사건을 서울고등법원으로 환송한다.\n\n【이유】\n\n원고의 상고이유를 판단한다.\n\n원심판결에 의하면 원심은 원고가 서울 ○○○구청 위생과 환경계장으로 재직하던 시기인 1975.7월부터 동 12월 사이에 원고의 부하직원이던 소외인이 관내 목욕탕 또는 이발관 등으로부터 단속을 빙자하여 금품을 받고 또는 무허가 드라

In [34]:
queries = [
    "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?",
]

rag_docs = ''
query_embeddings = model.encode_queries(queries)
dists, ids = index.search(query_embeddings.astype(np.float32), k=3)

print(f"query:\t{queries[0]}\n\n")
count = 0
for i in range(3):
    rag_docs += f"후보문장_{count}:\t{data_list[ids[0][i]]}\n"
    count +=1

print(rag_docs)

query:	지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?


후보문장_0:	{
    "판례일련번호": 93952,
    "사건명": "감봉처분취소",
    "사건번호": "78누164",
    "선고일자": "1978-08-22",
    "법원명": "대법원",
    "법원종류코드": null,
    "사건종류명": "일반행정",
    "사건종류코드": 400107.0,
    "판결유형": "판결",
    "선고": "선고",
    "판례상세링크": "/DRF/lawService.do?OC=pwshoot&target=prec&ID=93952&type=HTML&mobileYn=",
    "판시사항": "【판시사항】\n\n지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 당연히 감독의무를 위반하였다고 볼 수 있는지 여부",
    "판결요지": "【판결요지】\n\n지방동무원의 징계에 관한 어느 법규에도 부하직원에게 징계사유가 있다 하여 감독자가 감독의무를 위반한 것으로 본다는 규정이 없고 또 서울시의 지방공무원 징계의 양정에 관한 규칙의 취지도 감독자에게 지방공무원법상의 징계사유가 있음을 전체로 규정된 것이라 할 것이다",
    "참조조문": "【참조조문】\n\n서울특별시지방공무원징계의양정에관한규칙 제2조 제1항, 제2조 제3항",
    "참조판례": null,
    "판례내용": "【전문】\n\n【원고, 상고인】 원고\n\n【피고, 피상고인】 서울특별시장 소송대리인 변호사 장영복\n\n【원 판 결】 서울고등법원 1978.3.28. 선고 77구225 판결\n\n【주문】\n\n원판결을 파기하고 사건을 서울고등법원으로 환송한다.\n\n【이유】\n\n원고의 상고이유를 판단한다.\n\n원심판결에 의하면 원심은 원고가 서울 ○○○구청 위생과 환경계장으로 재직하던 시기인 1975.7월부터 동 12월 사이에 원고의 부하직원이던 소외인이 관내 목욕탕 또는 이발관 등으로부터 단속을 빙자하여 금품을 받고 또는 무허가 드

In [35]:
system_prompt = """You are a helpful assistant. And Answers must be in Korean. \
그리고 답변할땐 꼭 다음의 지시 사항을 준수해줘. \
1) 질문에 대한 답변 후보 문장들을 자세히 읽고 유저가 물어본 질문에 제시된 정보만 활용해서 질문과 정확성, 관련성, 신뢰성을 종합적으로 고려하여 답변을 만들어 제공해주세요. \
2) 그리고 후보문장 중 답변에 활용하는데 가장 적합한 후보가 어떤것 이였는지를 알려 주세요.\
3) 후보 문장들 중 질문에 대한 답이 없을 경우 "답변을 찾을 수 없습니다. 좀 더 구체적으로 질문해 주세요." 라고 답하세요."""

question = "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?"

user_prompt = question + "\
\n후보문장: \
" + rag_docs

print(user_prompt)

지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?
후보문장: 후보문장_0:	{
    "판례일련번호": 93952,
    "사건명": "감봉처분취소",
    "사건번호": "78누164",
    "선고일자": "1978-08-22",
    "법원명": "대법원",
    "법원종류코드": null,
    "사건종류명": "일반행정",
    "사건종류코드": 400107.0,
    "판결유형": "판결",
    "선고": "선고",
    "판례상세링크": "/DRF/lawService.do?OC=pwshoot&target=prec&ID=93952&type=HTML&mobileYn=",
    "판시사항": "【판시사항】\n\n지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 당연히 감독의무를 위반하였다고 볼 수 있는지 여부",
    "판결요지": "【판결요지】\n\n지방동무원의 징계에 관한 어느 법규에도 부하직원에게 징계사유가 있다 하여 감독자가 감독의무를 위반한 것으로 본다는 규정이 없고 또 서울시의 지방공무원 징계의 양정에 관한 규칙의 취지도 감독자에게 지방공무원법상의 징계사유가 있음을 전체로 규정된 것이라 할 것이다",
    "참조조문": "【참조조문】\n\n서울특별시지방공무원징계의양정에관한규칙 제2조 제1항, 제2조 제3항",
    "참조판례": null,
    "판례내용": "【전문】\n\n【원고, 상고인】 원고\n\n【피고, 피상고인】 서울특별시장 소송대리인 변호사 장영복\n\n【원 판 결】 서울고등법원 1978.3.28. 선고 77구225 판결\n\n【주문】\n\n원판결을 파기하고 사건을 서울고등법원으로 환송한다.\n\n【이유】\n\n원고의 상고이유를 판단한다.\n\n원심판결에 의하면 원심은 원고가 서울 ○○○구청 위생과 환경계장으로 재직하던 시기인 1975.7월부터 동 12월 사이에 원고의 부하직원이던 소외인이 관내 목욕탕 또는 이발관 등으로부터 단속을 빙자하여 금품을 받고 또는 무허가 드라이영

---
# Agent serving
  - 현재 터미널에서 nvidia-smi 를 수행해보면 BGE-M3 가 꽤 크게 gpu 메모리를 잡고 있습니다.
  - BGE-M3 모델도 내리고, faiss index 도 저장해주세요.

---

In [36]:
faiss.write_index(index, "law_index_faiss")

---
### kernel 재시작으로 모두 초기화  
  - jupyter lab 메뉴 > kernel > Restart Kernel
  - kernel 자체가 restart 됐기에, 위에 선언했던 import 내용과 정의한 함수도 모두 초기화 됐습니다.
  - vllm 으로 llama3.1 서빙

---

In [37]:
import torch
import gc

del tokenizer, model, raw_model  # 사용이 끝난 변수 삭제
gc.collect()  # 가비지 컬렉션 수동 호출
torch.cuda.empty_cache()  # 캐시된 메모리 해제

In [205]:
from openai import OpenAI
import math
import time
import json

client = OpenAI(
    base_url="http://localhost:8877/v1",
    api_key="token-abc123",
)

def vllm_generate(system_prompt, user_prompt):
    completion = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "user", "content": system_prompt + '\n' + user_prompt}
        ] if 'gemma' in model_name.lower() else [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        frequency_penalty=0.09,
        temperature=0.3,
        top_p=0.95,
        max_tokens=1024,
        stream=True,
    )

    final_answer = []
    assistant_response = ""

    start = time.time()

    # 스트림 모드에서는 completion.choices 를 반복문으로 순회
    for chunk in completion:
        chunk_content = chunk.choices[0].delta.content

        if isinstance(chunk_content, str):
            final_answer.append(chunk_content)
            # 토큰 단위로 실시간 답변 출력
            print(chunk_content, end="")
            assistant_response += chunk_content

    end = time.time()
    print(f"\n\ninference time: {end - start:.5f} sec \n\n")

In [39]:
from transformers import AutoTokenizer, AutoModel
import os, faiss
from FlagEmbedding import FlagModel
import numpy as np

model = FlagModel(
    'BAAI/bge-m3',
    query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
    use_fp16=True)

law_index = faiss.read_index("law_index_faiss")

In [40]:
queries = [
    "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?",
]

rag_docs = ''
query_embeddings = model.encode_queries(queries)
dists, ids = law_index.search(query_embeddings.astype(np.float32), k=3)

print(f"query:\t{queries[0]}\n\n")
count = 0
for i in range(3):
    rag_docs += f"후보문장_{count}:\t{data_list[ids[0][i]]}\n"
    count +=1

print(rag_docs)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


query:	지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?


후보문장_0:	{
    "판례일련번호": 93952,
    "사건명": "감봉처분취소",
    "사건번호": "78누164",
    "선고일자": "1978-08-22",
    "법원명": "대법원",
    "법원종류코드": null,
    "사건종류명": "일반행정",
    "사건종류코드": 400107.0,
    "판결유형": "판결",
    "선고": "선고",
    "판례상세링크": "/DRF/lawService.do?OC=pwshoot&target=prec&ID=93952&type=HTML&mobileYn=",
    "판시사항": "【판시사항】\n\n지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 당연히 감독의무를 위반하였다고 볼 수 있는지 여부",
    "판결요지": "【판결요지】\n\n지방동무원의 징계에 관한 어느 법규에도 부하직원에게 징계사유가 있다 하여 감독자가 감독의무를 위반한 것으로 본다는 규정이 없고 또 서울시의 지방공무원 징계의 양정에 관한 규칙의 취지도 감독자에게 지방공무원법상의 징계사유가 있음을 전체로 규정된 것이라 할 것이다",
    "참조조문": "【참조조문】\n\n서울특별시지방공무원징계의양정에관한규칙 제2조 제1항, 제2조 제3항",
    "참조판례": null,
    "판례내용": "【전문】\n\n【원고, 상고인】 원고\n\n【피고, 피상고인】 서울특별시장 소송대리인 변호사 장영복\n\n【원 판 결】 서울고등법원 1978.3.28. 선고 77구225 판결\n\n【주문】\n\n원판결을 파기하고 사건을 서울고등법원으로 환송한다.\n\n【이유】\n\n원고의 상고이유를 판단한다.\n\n원심판결에 의하면 원심은 원고가 서울 ○○○구청 위생과 환경계장으로 재직하던 시기인 1975.7월부터 동 12월 사이에 원고의 부하직원이던 소외인이 관내 목욕탕 또는 이발관 등으로부터 단속을 빙자하여 금품을 받고 또는 무허가 드

---
## 이때의 오류는 무엇을 의미 하나요?  
  - plain text  vs  embedding vector

---

In [41]:
directory_path = 'data_test/'
data_list = []
count = 0

# 디렉토리 내 모든 JSON 파일을 읽어들입니다.
for filename in os.listdir(directory_path):
    if filename.endswith(".json"):  # JSON 파일만 읽어들입니다.
        file_path = os.path.join(directory_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            # JSON 데이터를 텍스트 리스트 형태로 변환합니다.
            text_json = json.dumps(data, indent=4, ensure_ascii=False)
            data_list.append(text_json)
            if count % 10 ==0:
                print(". ", end="")
            count += 1
    else:
        continue

len(data_list)

. . . . . . . . . . . . . . . 

150

In [42]:
def rag_serch(query):
    queries = [ query ]

    rag_docs = ''
    query_embeddings = model.encode_queries(queries)
    dists, ids = law_index.search(query_embeddings.astype(np.float32), k=3)

    count = 0
    for i in range(3):
        rag_docs += f"후보문장_{count}:\t{data_list[ids[0][i]]}\n"
    count +=1

    return rag_docs

In [43]:
rag_serch("지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?")

'후보문장_0:\t{\n    "판례일련번호": 93952,\n    "사건명": "감봉처분취소",\n    "사건번호": "78누164",\n    "선고일자": "1978-08-22",\n    "법원명": "대법원",\n    "법원종류코드": null,\n    "사건종류명": "일반행정",\n    "사건종류코드": 400107.0,\n    "판결유형": "판결",\n    "선고": "선고",\n    "판례상세링크": "/DRF/lawService.do?OC=pwshoot&target=prec&ID=93952&type=HTML&mobileYn=",\n    "판시사항": "【판시사항】\\n\\n지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 당연히 감독의무를 위반하였다고 볼 수 있는지 여부",\n    "판결요지": "【판결요지】\\n\\n지방동무원의 징계에 관한 어느 법규에도 부하직원에게 징계사유가 있다 하여 감독자가 감독의무를 위반한 것으로 본다는 규정이 없고 또 서울시의 지방공무원 징계의 양정에 관한 규칙의 취지도 감독자에게 지방공무원법상의 징계사유가 있음을 전체로 규정된 것이라 할 것이다",\n    "참조조문": "【참조조문】\\n\\n서울특별시지방공무원징계의양정에관한규칙 제2조 제1항, 제2조 제3항",\n    "참조판례": null,\n    "판례내용": "【전문】\\n\\n【원고, 상고인】 원고\\n\\n【피고, 피상고인】 서울특별시장 소송대리인 변호사 장영복\\n\\n【원 판 결】 서울고등법원 1978.3.28. 선고 77구225 판결\\n\\n【주문】\\n\\n원판결을 파기하고 사건을 서울고등법원으로 환송한다.\\n\\n【이유】\\n\\n원고의 상고이유를 판단한다.\\n\\n원심판결에 의하면 원심은 원고가 서울 ○○○구청 위생과 환경계장으로 재직하던 시기인 1975.7월부터 동 12월 사이에 원고의 부하직원이던 소외인이 관내 목욕탕 또는 이발관 등으로부터 단속을 빙자하여 금품을 받고 또는 무허가 드라이영업행위를 적발하

In [44]:
question = "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?"
#question = "근무태만의 사유로 징계처분 받은것은 적합한가?"}

In [45]:
system_prompt = """You are a helpful assistant. And Answers must be in Korean. \
그리고 답변할땐 꼭 다음의 지시 사항을 준수해줘. \
1) 질문에 대한 답변 후보 문장들을 자세히 읽고 유저가 물어본 질문에 제시된 정보만 활용해서 질문과 정확성, 관련성, 신뢰성을 종합적으로 고려하여 답변을 만들어 제공해주세요. \
2) 후보문장은 json 형태로 돼 있습니다.
3) 특히 후보문장 내용 중 "판결요지" 내용들을 바탕으로 질문에 대한 답변을 정확하게 만들어 제공해주세요.
4) 후보문장 중 답변에 활용하는데 가장 적합한 후보가 어떤것 이였는지를 알려 주세요.\
5) 후보 문장들 중 질문에 대한 답이 없을 경우 "답변을 찾을 수 없습니다. 좀 더 구체적으로 질문해 주세요." 라고 답하세요."""

user_prompt =  question + "\n후보문장: " + rag_serch(question)
vllm_generate(system_prompt, user_prompt)

질문에 대한 답변 후보 문장 중 가장 적합한 문장은 후보문장_0 입니다.

**답변:**

지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는 것인가?

판결요지 내용에 따르면, 지방동무원의 징계에 관한 법규상 부하직원에게 징계사유가 있다 하더라도 감독자가 감독의무를 위반한 것으로 본다는 규정이 없으며, 서울시 지방공무원 징계의 양정에 관한 규칙의 취지도 감독자에게 지방공무원법상의 징계사유가 있음을 전제로 규정된 것이라고 합니다. 따라서 부하직원의 비위사실이 있는 경우, 감독자가 감독의무를 위반한 것으로 볼 수 없다는 것입니다.

**참고:** 다른 후보 문장들은 징계 처분의 유지가 가능하거나 재량권의 범위 내에서 징계가 가능하다는 점을 설명하고 있지만, 질문의 핵심인 “감독자의 위법 여부”에 대한 직접적인 답변을 제공하지는 않습니다.

inference time: 9.52921 sec 




---
## 생각해볼 topic  
  - json 덩어리 전체가 embedding 됐는데, 이것이 과연 question 을 embedding 한것과 비교하는것이 적합한것인가?
  - 지금처럼 전처리가 잘 돼 있는 데이터라면, 추가로 시도해볼만한 것은 어떤것이 있을까요?
---

In [46]:
from datetime import datetime
now = datetime.now()
that_time = datetime.strptime('1991-11-22', '%Y-%m-%d')
time_gap = (now - that_time).days // 30

print('두 날짜의 차')
print(time_gap, '주')

두 날짜의 차
407 주


In [206]:
import json
from datetime import datetime
# Tool 의 동작 내용을 컴퓨터 언어로 정의 (우리가 이미 잘 알고 있는 python code)

def get_calc_month(date):
    print("판례의 날짜 ({})".format(date))
    if date is None:
      return json.dumps({"date": "정보 부족"})

    now = datetime.now()
    that_time = datetime.strptime(date, '%Y-%m-%d')
    time_gap = (now - that_time).days // 30

    send_info = {
        "time_gap": time_gap,
    }
    print(send_info)
    print("--------------------")
    return json.dumps(send_info)

# Tool 의 동작 내용을 포맷에 맞추어 인간의 언어로 정의 (LLM에게 전달할 내용)
tool_choice = {
    "function": {
        "name": "get_calc_month"
    },
    "type": "function"
}

tools = [{
    "type": "function",
    "function": {
        "name": "get_calc_month",
        "description": "Jsonified data format date calculataion how many months have passed since a given date.",
        "parameters": {
            "type": "object",
            "properties": {
                # "time": {
                #     "type": "string",
                #     "description": 'eg. 선고된 날짜 현재부터 35개월 전, 35개월 전...',
                # },
                "date": {
                    "type": "string",
                    "description": '%Y-%m-%d (e.g. 1991-11-22, 2004-04-28, 2023-12-12, 1942-03-12, 1999-10-31...)',
                },
                "answer": {
                    "type": "string",
                    "description": '요청사항에 따른 응답.',
                },
            },
            "required": ["time", "date", "answer"],
        },
    },
}]

In [207]:
def vllm_generate_tool(system_prompt, user_prompt):
    chat_history =[
            {"role": "user", "content": system_prompt + '\n' + user_prompt}
        ] if 'gemma' in model_name.lower() else [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]

    tool_calls_stream  = client.chat.completions.create(
        model=model_name,
        messages=chat_history,
        tools=tools,
        tool_choice=tool_choice,
        frequency_penalty=0.09,
        temperature=0.1,
        top_p=0.9,
        max_tokens=1024,
        top_logprobs=19,
        logprobs=True,
        stream=False,
    )

    tool_call = tool_calls_stream.choices[0].message.tool_calls
    function_response = ""

    if tool_calls_stream.choices[0].message.tool_calls[0].function != None:

        tool_call = tool_calls_stream.choices[0].message.tool_calls[0]

        if tool_call.function:
            if tool_call.function.name:
                print(f"tool call name: {tool_call.function.name}")

            if tool_call.function.arguments:
                print(f"{tool_call.function.name}: {tool_call.function.arguments}")

        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        # tool 호출
        if function_name == "get_calc_month":
            function_response = get_calc_month( date=function_args.get("date"))

    chat_history.append(
        {
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )

    second_completion = client.chat.completions.create(
        model=model_name,
        messages=chat_history,
        frequency_penalty=0.09,
        temperature=0.1,
        top_p=0.9,
        max_tokens=1024,
        stream=True,
    )

    final_answer = []
    assistant_response = ""

    start = time.time()

    # 스트림 모드에서는 completion.choices 를 반복문으로 순회
    for chunk in second_completion:
        chunk_content = chunk.choices[0].delta.content

        if isinstance(chunk_content, str):
            final_answer.append(chunk_content)
            # 토큰 단위로 실시간 답변 출력
            print(chunk_content, end="")
            assistant_response += chunk_content

    end = time.time()
    print(f"\n\ninference time: {end - start:.5f} sec \n\n")

In [208]:
question = "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?"

In [209]:
system_prompt = """You are a helpful assistant. And Answers must be in Korean.
tool을 사용하여 정보를 찾고 마지막 응답을 답변할땐 꼭 다음의 지시 사항을 준수해줘.
1) 질문에 대한 답변 후보 문장들을 자세히 읽고 유저가 물어본 질문에 제시된 정보만 활용해서 질문과 정확성, 관련성, 신뢰성을 종합적으로 고려하여 답변을 만들어 제공해주세요.
2) 후보문장은 json 형태로 돼 있습니다.
3) 후보문장 내용 중 "판결요지" 내용들을 바탕으로 질문에 대한 답변을 정확하게 만들어 제공해주세요.
4) 후보문장 중 답변에 활용하는데 가장 적합한 후보가 어떤것 이였는지를 알려 주세요.
5) 특히 선택된 후보문장 중 "선고일자" 를 활용해서 현재부터의 날짜 차이를 개월수로 꼭 표현해줘. ex) 선고된 날짜 현재부터 35개월 전
6) 후보 문장들 중 질문에 대한 답이 없을 경우 "답변을 찾을 수 없습니다. 좀 더 구체적으로 질문해 주세요." 라고 답하세요."""

user_prompt =  question + "\n후보문장: " + rag_serch(question)

vllm_generate_tool(system_prompt, user_prompt)

tool call name: get_calc_month
get_calc_month: {
  "date": "1986-11-25",
  "answer": "원심은 원고가 지방건축기원보로 임명되어 서울특별시 민방위국 비상계획과에 근무하면서 직상급자인 소외 1의 위법, 부당처리를 알고도 소외 1의 부당한 지시에 따라 군협의 결과 \"건축불가\"사항을 임의로 \"건축가능\"하다고 회시하고 또 군부대와의 협의 없이 건축가능여부에 대하여 임의로 처리한 사실을 인정한 다음 위와 같은 사유는 국가공무원법 제78조 제1항 제1호, 제2호, 제56조의 소정의 직무상의 의무위반 및 성실의무위반에 해당하는 것으로서 징계사유가 된다고 본 것이나, 원심은 원고가 직속상관인 소외 1의 처리지시를 믿고 그대로 따른 것으로서 이로 인한 금품수수등의 행위도 없었던 사실을 인정한 다음 징계재량의 범위를 일탈하여 한 위법한 처분이라고 판시하고 있는바 기록에 비추어 볼 때 원심의 위와 같은 인정과 판단은 수긍이 되고 거기에 논지와 같이 형평의 원칙에 반하거나 재량권에 관한 법리를 오해하는등 위법이 있다고 할 수 없다.\n\n따라서 상고를 기각하고, 상고 소송비용은 패소자의 부담으로 하기로 관여법관의 의견이 일치되어 주문과 같이 판결한다.\n\n대법관   김달식(재판장) 정기승 박우동"}
판례의 날짜 (1986-11-25)
{'time_gap': 467}
--------------------
알겠습니다. 답변을 제공해 드리겠습니다.

inference time: 0.14009 sec 


